# Ingest CXG spatial h5ad files from a directory and create Census object

## Create a manifest file containing the location of spatial datasets to load

In [1]:
import os

from cellxgene_census_builder.build_soma.manifest import load_manifest

In [2]:
def create_manifest_csv_file(spatial_datasets_dir, manifest_file_path):
    file_ids = [os.path.splitext(filename)[0] for filename in os.listdir(spatial_datasets_dir)]
    file_paths = [os.path.join(spatial_datasets_dir, filename) for filename in os.listdir(spatial_datasets_dir)]
    manifest_content = "\n".join([", ".join(pair) for pair in zip(file_ids, file_paths, strict=False)])

    with open(manifest_file_path, "w") as f:
        f.write(manifest_content.strip())

In [3]:
# NOTE: Replace value of `spatial_datasets_dir` with a valid path to a folder containing datasets
spatial_datasets_dir = "/Users/psridharan/code/cellxgene-census/ps_stuff/spatial_test_datasets"
manifest_file_path = "./manifest.csv"
blocklist_file_path = "./blocklist.txt"

create_manifest_csv_file(spatial_datasets_dir, manifest_file_path)

In [4]:
datasets = load_manifest(manifest_file_path, blocklist_file_path)

In [5]:
datasets

[Dataset(dataset_id='c6f6e674-b59d-46cf-8525-73f64f9eef8c', dataset_asset_h5ad_uri='/Users/psridharan/code/cellxgene-census/ps_stuff/spatial_test_datasets/c6f6e674-b59d-46cf-8525-73f64f9eef8c.h5ad', dataset_version_id='', dataset_h5ad_path='', dataset_title='', citation='', collection_id='', collection_name='', collection_doi='', asset_h5ad_filesize=-1, cell_count=-1, mean_genes_per_cell=-1.0, schema_version='', dataset_total_cell_count=0, soma_joinid=-1),
 Dataset(dataset_id='fa3893cb-d420-42ac-8263-09719a26102e', dataset_asset_h5ad_uri='/Users/psridharan/code/cellxgene-census/ps_stuff/spatial_test_datasets/fa3893cb-d420-42ac-8263-09719a26102e.h5ad', dataset_version_id='', dataset_h5ad_path='', dataset_title='', citation='', collection_id='', collection_name='', collection_doi='', asset_h5ad_filesize=-1, cell_count=-1, mean_genes_per_cell=-1.0, schema_version='', dataset_total_cell_count=0, soma_joinid=-1),
 Dataset(dataset_id='07998bf8-d070-41bb-a584-f8bdd1193aef', dataset_asset_h5ad